In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer
from transformers import BertModel
from transformers import RobertaModel
from transformers import DistilBertModel
import torch
import torch.nn as nn

In [4]:
from transformers import AutoTokenizer
from transformers import BertModel
from transformers import RobertaModel
from transformers import DistilBertModel
import torch
import torch.nn as nn

c:\Users\fedor\KC_Final_RecSys\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine


pg_user = os.getenv("POSTGRES_USER", " ")
pg_password = os.getenv("POSTGRES_PASSWORD", " ")
pg_host = os.getenv("POSTGRES_HOST", " ")
pg_port = os.getenv("POSTGRES_PORT", " ")
pg_db = os.getenv("POSTGRES_DATABASE", " ")

engine = create_engine(
    f"postgresql://{pg_user}:{pg_password}"
    f"{pg_host}:{pg_port}/{pg_db}"
)

post_text_df = pd.read_sql('SELECT * FROM public.post_text_df;', con=engine) # считываем таблицу
post_text_df

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [3]:
texts = post_text_df["text"]
texts

0       UK economy facing major risks\n\nThe UK manufa...
1       Aids and climate top Davos agenda\n\nClimate c...
2       Asian quake hits European shares\n\nShares in ...
3       India power shares jump on debut\n\nShares in ...
4       Lacroix label bought by US firm\n\nLuxury good...
                              ...                        
7018    OK, I would not normally watch a Farrelly brot...
7019    I give this movie 2 stars purely because of it...
7020    I cant believe this film was allowed to be mad...
7021    The version I saw of this film was the Blockbu...
7022    Piece of subtle art. Maybe a masterpiece. Doub...
Name: text, Length: 7023, dtype: object

In [8]:
def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }
    
    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [9]:
tokenizer, model = get_model('bert')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

c:\Users\fedor\KC_Final_RecSys\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fedor\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download

cuda:0
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [5]:
def tokenization(example):
    return tokenizer.batch_encode_plus(example["text"], add_special_tokens = True, return_token_type_ids = False, truncation = True)

In [11]:
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from transformers import DataCollatorWithPadding
from torch.utils.data import Dataset

In [12]:
class TextsDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.encodings = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens = True,
            return_token_type_ids = False,
            truncation = True,
            padding=True,
            return_tensors='pt')

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings['input_ids'][idx],
            "attention_mask": self.encodings['attention_mask'][idx]
            }



In [18]:

dataset = TextsDataset(texts.values.tolist(), tokenizer)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
loader = DataLoader(dataset, batch_size=2, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [19]:
model = model.to(device)

In [20]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [21]:
from tqdm import tqdm
def get_embeddings_labels(model, loader):
    model.eval()
    total_embeddings = []

    with torch.no_grad():
        for batch in tqdm(loader):

            batch = {key: val.to(device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
            outputs = model(**batch)
            embeddings = outputs.last_hidden_state[:, 0, :]  # CLS токен
            total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [22]:
post_embeddings = get_embeddings_labels(model, loader)

  0%|          | 0/3512 [00:00<?, ?it/s]c:\Users\fedor\KC_Final_RecSys\venv\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 3512/3512 [05:39<00:00, 10.34it/s]


In [23]:
post_embeddings.shape

torch.Size([7023, 768])

In [24]:
post_embeddings[0]

tensor([ 1.4036e-01, -1.4070e-01, -5.7568e-01, -1.1817e-01, -3.1532e-01,
        -1.1438e-01,  4.3198e-01, -1.4429e-01,  3.2331e-03, -1.1878e+00,
        -3.3517e-01,  8.5946e-02, -1.0785e-01, -3.6509e-01, -3.5554e-02,
         1.1618e-01,  2.9538e-01, -6.3872e-02,  6.1660e-02, -4.6042e-01,
        -7.2604e-02, -3.3956e-01,  2.6281e-01, -3.3909e-01,  2.1391e-02,
        -1.2962e-01,  6.0904e-02,  3.6134e-01, -2.9707e-01,  7.4856e-01,
         1.0701e-01,  2.3395e-01, -3.4371e-01,  1.8407e-01,  4.9053e-02,
        -1.5357e-01,  2.0610e-02, -3.9094e-01, -6.3100e-02,  9.3924e-02,
        -2.2694e-01,  5.5329e-01,  2.8346e-01, -2.8457e-01,  2.5225e-02,
        -2.1027e-01,  3.0637e-01,  1.4088e-02,  2.5683e-03,  2.2604e-02,
        -1.5580e-01, -7.6782e-04,  1.3047e-01,  6.4576e-02, -1.3793e-01,
         1.8241e-01,  2.8353e-01,  1.1229e-01, -6.3608e-01,  5.5540e-01,
         3.0667e-01, -4.7828e-01,  1.1780e-01,  9.7994e-02,  1.9741e-01,
         2.6836e-01, -8.4358e-02, -4.3326e-02, -3.6

In [27]:
texts_df = pd.DataFrame(post_embeddings)
texts_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.140363,-0.140695,-0.575681,-0.118175,-0.315325,-0.114379,0.431976,-0.144286,0.003233,-1.187819,...,0.051767,0.614535,-0.613107,0.131061,0.202857,0.175932,-0.167769,-0.137880,0.042959,0.142284
1,0.157530,-0.097739,-0.230651,-0.364431,-0.242782,0.310065,0.374489,-0.089235,0.202152,-1.130711,...,0.586031,0.652154,-0.112308,-0.085171,-0.051813,0.240048,0.200298,-0.300891,0.190542,0.019754
2,0.314568,-0.115163,-0.181322,-0.274697,-0.357378,0.285228,0.266597,0.002641,-0.033905,-1.092078,...,0.416194,0.641697,-0.326962,-0.042802,-0.073850,0.212023,-0.090192,-0.354050,-0.204324,-0.027024
3,0.415117,-0.241302,-0.260732,-0.436026,-0.194695,0.130078,0.458805,-0.235223,-0.032936,-1.008514,...,0.791115,0.562938,-0.194189,0.022461,0.108904,0.019627,0.362089,-0.150885,-0.048834,0.083070
4,0.614585,-0.235812,-0.047733,-0.406701,-0.284798,0.124150,0.545585,-0.284447,0.047563,-1.139113,...,0.605897,0.518612,0.007372,0.032436,0.015633,0.055697,0.145704,-0.061322,-0.021119,0.121545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,0.521686,0.292126,-0.210760,-0.219636,0.124439,-0.032428,0.055144,0.118430,-0.033624,-1.167992,...,0.575904,0.319888,-0.427476,0.161139,0.179984,0.053670,0.051578,-0.353712,-0.010725,-0.047067
7019,0.487033,0.442943,-0.251731,-0.303212,0.068587,-0.176149,0.235078,-0.050546,-0.011601,-1.161575,...,0.077758,0.174062,-0.359764,-0.196519,0.043920,0.178851,-0.054636,-0.233470,0.349205,-0.031997
7020,0.619477,0.274618,-0.126504,-0.110225,0.167652,-0.184623,0.244475,0.020286,0.058746,-1.046321,...,0.307723,0.187839,-0.378197,-0.223615,0.019644,0.250250,0.038904,-0.358137,-0.164277,0.171024
7021,0.694089,0.067175,-0.228680,-0.255529,0.134818,0.223566,0.249515,-0.032107,-0.160485,-1.041767,...,0.361885,0.352623,-0.307157,-0.139961,0.150053,0.015053,0.045805,0.037858,0.141017,0.124412


In [28]:
from sklearn.decomposition import PCA

pca = PCA(n_components=30)
posts_pca = pca.fit_transform(texts_df)
posts_pca = pd.DataFrame(posts_pca)
posts_pca.columns = [f"PCA_{i}" for i in range(posts_pca.shape[1])]
posts_pca

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,...,PCA_20,PCA_21,PCA_22,PCA_23,PCA_24,PCA_25,PCA_26,PCA_27,PCA_28,PCA_29
0,0.954858,-1.755624,0.206988,1.697693,2.381051,0.014761,-0.139383,0.573181,2.037544,-0.976322,...,0.335726,-0.183577,-0.195586,0.566892,-0.526097,-0.288334,-0.196174,-0.482868,-0.792480,-0.202706
1,3.082154,-0.872370,-1.121488,0.695178,-0.051383,0.234757,-0.309028,0.000403,0.246722,0.531314,...,-1.031039,0.207809,-0.431815,-0.036251,0.748353,0.444547,-0.212094,0.339106,0.004274,0.162952
2,2.298748,-0.771819,-1.484293,0.921830,-0.043767,0.577127,0.102362,0.678321,1.164003,-1.001129,...,0.148509,0.148171,-0.024870,-0.075873,0.176585,-0.884598,-0.847778,0.175247,-0.402457,0.345049
3,3.830454,-0.031844,-1.307996,-2.101641,-0.484527,-0.095325,0.206708,-0.808239,-0.210456,0.079055,...,0.202360,0.090817,-0.308733,0.109618,-0.085470,-0.037503,0.355612,-0.208097,-0.078597,0.181073
4,2.248866,0.231330,-1.637729,-1.685148,-0.175614,-0.363434,0.877843,-0.201348,-0.476321,0.037071,...,0.085243,0.336461,0.257457,-0.191238,-0.755810,0.078563,0.088131,-0.212274,-0.178781,-0.153670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,-2.656974,-1.065918,-1.688349,-0.109286,-0.562750,-0.528287,0.269984,0.190370,-1.075474,-0.433447,...,-0.561882,0.112290,-0.115465,-0.405388,-0.146213,0.370069,-0.392105,-0.523010,0.364300,-0.412168
7019,-2.547192,-0.549885,-0.247586,0.458581,-0.235308,-0.913795,0.242977,0.429372,0.440350,-0.022954,...,0.474414,-0.310578,0.348599,0.378550,-0.639824,0.761652,-0.272780,-0.485797,0.149530,0.413217
7020,-2.438389,-0.878828,-1.034504,-0.249473,-0.589287,-0.427735,0.258929,0.983647,-0.138929,-0.731653,...,-0.286017,-1.029164,-0.111825,0.216974,0.127558,0.009201,0.302979,-0.002441,-0.104451,-0.188348
7021,-2.499129,-1.253236,-0.025754,-0.992780,0.589764,0.471234,0.410792,-0.301116,-0.129267,-0.344709,...,-0.124784,0.215337,-0.418270,-0.252481,0.023823,-0.201580,0.297915,0.467324,0.324102,0.224584


In [29]:
post_data = pd.concat([post_text_df, posts_pca], axis=1)
post_data

,post_id,text,topic,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,...,PCA_20,PCA_21,PCA_22,PCA_23,PCA_24,PCA_25,PCA_26,PCA_27,PCA_28,PCA_29
0,1,UK economy facing major risks\n\nThe UK manufa...,business,0.954858,-1.755624,0.206988,1.697693,2.381051,0.014761,-0.139383,...,0.335726,-0.183577,-0.195586,0.566892,-0.526097,-0.288334,-0.196174,-0.482868,-0.792480,-0.202706
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,3.082154,-0.872370,-1.121488,0.695178,-0.051383,0.234757,-0.309028,...,-1.031039,0.207809,-0.431815,-0.036251,0.748353,0.444547,-0.212094,0.339106,0.004274,0.162952
2,3,Asian quake hits European shares\n\nShares in ...,business,2.298748,-0.771819,-1.484293,0.921830,-0.043767,0.577127,0.102362,...,0.148509,0.148171,-0.024870,-0.075873,0.176585,-0.884598,-0.847778,0.175247,-0.402457,0.345049
3,4,India power shares jump on debut\n\nShares in ...,business,3.830454,-0.031844,-1.307996,-2.101641,-0.484527,-0.095325,0.206708,...,0.202360,0.090817,-0.308733,0.109618,-0.085470,-0.037503,0.355612,-0.208097,-0.078597,0.181073
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,2.248866,0.231330,-1.637729,-1.685148,-0.175614,-0.363434,0.877843,...,0.085243,0.336461,0.257457,-0.191238,-0.755810,0.078563,0.088131,-0.212274,-0.178781,-0.153670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,-2.656974,-1.065918,-1.688349,-0.109286,-0.562750,-0.528287,0.269984,...,-0.561882,0.112290,-0.115465,-0.405388,-0.146213,0.370069,-0.392105,-0.523010,0.364300,-0.412168
7019,7316,I give this movie 2 stars purely because of it...,movie,-2.547192,-0.549885,-0.247586,0.458581,-0.235308,-0.913795,0.242977,...,0.474414,-0.310578,0.348599,0.378550,-0.639824,0.761652,-0.272780,-0.485797,0.149530,0.413217
7020,7317,I cant believe this film was allowed to be mad...,movie,-2.438389,-0.878828,-1.034504,-0.249473,-0.589287,-0.427735,0.258929,...,-0.286017,-1.029164,-0.111825,0.216974,0.127558,0.009201,0.302979,-0.002441,-0.104451,-0.188348
7021,7318,The version I saw of this film was the Blockbu...,movie,-2.499129,-1.253236,-0.025754,-0.992780,0.589764,0.471234,0.410792,...,-0.124784,0.215337,-0.418270,-0.252481,0.023823,-0.201580,0.297915,0.467324,0.324102,0.224584


In [31]:
post_data.to_sql(
    "fedorrybalov_lesson_22_post_data_DL",
    con=engine,
    if_exists="replace",
    index=False,
    chunksize=500  # или 1000
)

7023